In [1]:
from data.Safe2Unsafe import DeepAccidentDataset
from method.dynamics import HyperplaneEncoder
from method.barriers import DiscriminatingHyperplane
from method.trainers import HyperplaneTrainer
import torch
import time
import pytorch_lightning as pl
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [4]:
data = DeepAccidentDataset(train_batch_size=32,val_batch_size=32,num_workers=16)
data.setup()
train_dataloader = data.train_dataloader()
test_dataloader = data.val_dataloader()

In [2]:
latent_dim = 16
barrier = DiscriminatingHyperplane(2,latent_dim=latent_dim)
# model = HyperplaneEncoder(2,"cuda",model="google/vit-base-patch16-224",latent_dim=latent_dim)
model = HyperplaneEncoder(2,"cuda",model="vc1",latent_dim=latent_dim)
# model = HyperplaneEncoder(2,"cuda",model="openai/clip-vit-base-patch16",latent_dim=latent_dim)
# model = HyperplaneEncoder(2,"cuda",model="resnet",latent_dim=latent_dim)
trainer = HyperplaneTrainer(model,barrier)
checkpoint = torch.load("/root/tf-logs/DiscriminatingHyperplane/version_4/checkpoints/last.ckpt")
trainer.load_state_dict(checkpoint['state_dict'])
trainer.to("cuda")

HyperplaneTrainer(
  (model): HyperplaneEncoder(
    (encoder): ViTAttentionEncoder(
      (ViT): VisionTransformer(
        (patch_embed): PatchEmbed(
          (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
          (norm): Identity()
        )
        (pos_drop): Dropout(p=0.0, inplace=False)
        (norm_pre): Identity()
        (blocks): Sequential(
          (0): Block(
            (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (attn): Attention(
              (qkv): Linear(in_features=768, out_features=2304, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=768, out_features=768, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): Identity()
            (drop_path1): Identity()
            (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=768, out_featu

In [5]:
b_all = []
label_all = []
trainer.eval()
for idx, (i,u,label) in enumerate(test_dataloader):
    i, u = i.to("cuda"), u.to("cuda")
    x = trainer.model.simulate(i,u)
    a,b = trainer.barrier(x)
    value = torch.einsum("btc,btc->bt",a,u) + b
    b_all.append((value < 0).cpu())
    label_all.append(label.squeeze(-1))
import numpy as np
bs = torch.cat(b_all)
labels = torch.cat(label_all)
results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results_hyperplane_vc1.txt",results)

100%|██████████| 4/4 [00:00<00:00,  5.04it/s]


In [8]:
results = np.loadtxt("./results_hyperplane_vc1.txt")
regular = results[results[:,-1] == 0,:-1]
collision = results[results[:,-1] == 1,:-1]
acc_regular = (regular <= 0).mean()
acc_collision = (collision > 0).mean()
print(acc_regular,acc_collision)

0.9958940397350994 0.017777777777777778


In [7]:
regular

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       ...,
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])